In [5]:
import json
import sys
import os
import cv2
import numpy as np

sys.path.insert(0, '../../')
from data_loader import get_data_to_load, update_data_to_load


files = get_data_to_load(loading_file='../03_mapping/data_list', image_file_location='../../1_data_collection/data/', allow_new_file_creation=False)

images = list(filter(lambda x: x.endswith('.png'), files))

filtered_files = []

for image_path in images:
   # Load an image
  image = cv2.imread(image_path)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  # Compute means and variances
  variance_colors = np.var(image, axis=(0, 1))
  
  max_variance = np.max(variance_colors)
  
  if max_variance > 50:
    filtered_files.append(image_path)
  else:
    print("Too low max RGB variance:", image_path)
    print("Max RGB variance:", max_variance)

print(len(files))

files = update_data_to_load(files, old_loading_file='../03_mapping/data_list', file_location='../01_enriching/data/', image_file_location='../../1_data_collection/data/')

4
